In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from typing import Literal
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage

In [3]:
from langgraph.graph import END, START, StateGraph, MessagesState
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

In [4]:
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."

In [5]:
tools = [get_weather]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

In [6]:
def call_model(state: MessagesState):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}


def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [7]:
workflow = StateGraph(MessagesState)
tool_node = ToolNode(tools)

In [8]:
workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", "agent")

graph = workflow.compile(checkpointer=MemorySaver())

In [9]:
graph.invoke(
    {"messages": [HumanMessage(content="How is the weather in munich?")]},
    config={"configurable": {"thread_id": 1}},
)

{'messages': [HumanMessage(content='How is the weather in munich?', additional_kwargs={}, response_metadata={}, id='6ce58159-5aa3-460f-93d7-8bc2aab1d597'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_g8MI8uPLaqB9ZxTyNGtYlx9Y', 'function': {'arguments': '{"location":"Munich"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 52, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7f375d0a-c4e1-4188-8350-f0838d0d4b4e-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Munich'}, 'id': 'call_g8MI8uPLaqB9ZxTyNGtYlx9Y', 'type': 'tool_call'}], usage_metadata={

In [10]:
graph.invoke(
    {
        "messages": [
            HumanMessage(content="What would you recommend to do in that city then?")
        ]
    },
    config={"configurable": {"thread_id": 1}},
)

{'messages': [HumanMessage(content='How is the weather in munich?', additional_kwargs={}, response_metadata={}, id='6ce58159-5aa3-460f-93d7-8bc2aab1d597'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_g8MI8uPLaqB9ZxTyNGtYlx9Y', 'function': {'arguments': '{"location":"Munich"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 52, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7f375d0a-c4e1-4188-8350-f0838d0d4b4e-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Munich'}, 'id': 'call_g8MI8uPLaqB9ZxTyNGtYlx9Y', 'type': 'tool_call'}], usage_metadata={

### Getting production ready - async and streaming

In [ ]:
# Not all LLMs can be streamed but openai can.

model = ChatOpenAI(model="gpt-4o-mini", streaming=True).bind_tools(tools)

In [11]:
# ainvoke and await for async

async def call_model(state: MessagesState):
    messages = state["messages"]
    response = await model.ainvoke(messages)
    return {"messages": [response]}

In [12]:
workflow = StateGraph(MessagesState)
tool_node = ToolNode(tools)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {"tools": "tools", END: END},
)
workflow.add_edge("tools", "agent")

graph = workflow.compile(checkpointer=MemorySaver())

In [13]:
inputs = {"messages": [HumanMessage(content="How is the weather in Munich?")]}
config = {"configurable": {"thread_id": 2}}

In [14]:
# Must use await 

await graph.ainvoke(input=inputs, config=config)

{'messages': [HumanMessage(content='How is the weather in Munich?', additional_kwargs={}, response_metadata={}, id='e9145658-91fa-4db2-af1c-e37e605a9a82'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_j0T8qIyoz6bCWnUGHKxWg0kU', 'function': {'arguments': '{"location":"Munich"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 51, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bd1f4ac3-56c7-49e6-988b-0e1f56c9c976-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Munich'}, 'id': 'call_j0T8qIyoz6bCWnUGHKxWg0kU', 'type': 'tool_call'}], usage_metadata={

# 2 ways to stream for 2 different purposes

In [15]:
# This is streaming that basically shows steps and not recommended for UI like chatbots

inputs = {"messages": [HumanMessage(content="How is the weather in Munich?")]}
async for output in graph.astream(inputs, stream_mode="updates", config=config):
    # stream_mode="updates" yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value["messages"][-1].pretty_print())

Output from node 'agent':
---
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_GZH7P3Txo9IotMsr1WrzPpvz)
 Call ID: call_GZH7P3Txo9IotMsr1WrzPpvz
  Args:
    location: Munich
None
Output from node 'tools':
---
================================= Tool Message =================================
Name: get_weather

It's 15 degrees Celsius and cloudy.
None
Output from node 'agent':
---
================================== Ai Message ==================================

The weather in Munich is currently 15 degrees Celsius and cloudy.
None


In [16]:
from langchain_core.messages import AIMessageChunk, HumanMessage

inputs = [HumanMessage(content="How is the weather in Munich?")]
gathered = None

async for msg, metadata in graph.astream(
    {"messages": inputs}, stream_mode="messages", config=config
):
    if msg.content and not isinstance(msg, HumanMessage):
        # Print each token as it streams in
        print(msg.content, end="|", flush=True)

    # Handle the AI message chunks for proper assembly
    if isinstance(msg, AIMessageChunk):
        if gathered is None:
            gathered = msg
        else:
            gathered = gathered + msg

It's 15 degrees Celsius and cloudy.|The| weather| in| Munich| is| currently| |15| degrees| Celsius| and| cloudy|.|

In [18]:
print(gathered.content)

The weather in Munich is currently 15 degrees Celsius and cloudy.
